In [4]:
%matplotlib inline

import utils_ted
from utils_ted import *

In [5]:
model_path = 'data/imdb/models/'
if not os.path.exists(model_path): os.makedirs(model_path)

In [6]:
batch_size = 64

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [7]:
from keras.datasets import imdb

In [8]:
idx = imdb.get_word_index()
len(idx)

88584

This is the word list:

In [9]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [10]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code copied from keras.datasets:

In [11]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

In [12]:
print(len(x_train))
print(len(x_test))

25000
25000


Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [13]:
#x_train[0][:3]
", ".join(map(str, x_train[0]))

'1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32'

The first word of the first review is 1. Let's see what that is.

In [14]:
idx2word[1]

'the'

In [15]:
' '.join([idx2word[o] for o in x_train[0]])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

In [16]:
# Ted own implementation
def num2word(x): return idx2word[x]
" ".join(map(num2word, x_train[0]))

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

The labels are 1 for positive, 0 for negative.

In [17]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

Reduce vocab size by setting rare words to max index.

In [18]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s ])for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s ])for s in x_test]

Look at distribution of lengths of sentences.

In [19]:
len(trn)

25000

In [20]:
#lens = np.array(map(len, trn))
lens = np.array([len(x) for x in trn])
(lens.max(), lens.min(), lens.mean())

(2494, 11, 238.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [21]:
from keras.preprocessing import sequence

In [22]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0.)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0.)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [23]:
print(trn.shape)
print(test.shape)

(25000, 500)
(25000, 500)


## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [24]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')
])

In [25]:
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(trn, y_train, validation_data=(test, y_test), batch_size=batch_size, epochs=2, verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
27s - loss: 0.4759 - acc: 0.7398 - val_loss: 0.3135 - val_acc: 0.8640
Epoch 2/2
25s - loss: 0.2071 - acc: 0.9228 - val_loss: 0.3058 - val_acc: 0.8729


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [27]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  


In [28]:
conv1.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
conv1.fit(trn, y_train, validation_data=(test, y_test), batch_size=batch_size, epochs=2, verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
107s - loss: 0.5331 - acc: 0.6959 - val_loss: 0.3066 - val_acc: 0.8751
Epoch 2/2
106s - loss: 0.2976 - acc: 0.8939 - val_loss: 0.2602 - val_acc: 0.8889


In [30]:
conv1.save_weights(model_path+'conv1.h5')

In [31]:
conv1.load_weights(model_path+'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [32]:
def load_glove(loc):
    return (load_array(loc+'.dat'), 
            pickle.load(open(loc+'_words.pkl', 'rb')),
            pickle.load(open(loc+'_wordidx.pkl', 'rb')))

In [33]:
glove_path = 'data/glove.6B/results/'
vecs, words, wordidx = load_glove(glove_path+'50d')

In [34]:
vecs.shape

(400000, 50)

In [35]:
from keras.utils import get_file

In [36]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    #%mkdir -p $glove_path
    if not os.path.exists(glove_path): os.makedirs(glove_path)
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [37]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'), encoding='utf8'),
        pickle.load(open(loc+'_idx.pkl','rb'), encoding='utf8'))

In [38]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [39]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    
    for i in range(1, len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))
            #emb[i] = normal(scale=1, size=(n_fact,))
    
    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    #emb[-1] = normal(scale=1, size=(n_fact,))
    emb = emb / 3  #??
    return emb

In [40]:
emb = create_emb()

In [41]:
emb.shape

(5000, 50)

In [42]:
print(idx2word[1])
print(wordidx[idx2word[1]])
print(vecs[wordidx[idx2word[1]]])
print(vecs[wordidx[idx2word[1]]].shape)
#emb[:3,:]

the
0
[  4.18000013e-01   2.49679998e-01  -4.12420005e-01   1.21699996e-01
   3.45270008e-01  -4.44569997e-02  -4.96879995e-01  -1.78619996e-01
  -6.60229998e-04  -6.56599998e-01   2.78430015e-01  -1.47670001e-01
  -5.56770027e-01   1.46579996e-01  -9.50950012e-03   1.16579998e-02
   1.02040000e-01  -1.27920002e-01  -8.44299972e-01  -1.21809997e-01
  -1.68009996e-02  -3.32789987e-01  -1.55200005e-01  -2.31309995e-01
  -1.91809997e-01  -1.88230002e+00  -7.67459989e-01   9.90509987e-02
  -4.21249986e-01  -1.95260003e-01   4.00710011e+00  -1.85939997e-01
  -5.22870004e-01  -3.16810012e-01   5.92130003e-04   7.44489999e-03
   1.77780002e-01  -1.58969998e-01   1.20409997e-02  -5.42230010e-02
  -2.98709989e-01  -1.57490000e-01  -3.47579986e-01  -4.56370004e-02
  -4.42510009e-01   1.87849998e-01   2.78489990e-03  -1.84110001e-01
  -1.15139998e-01  -7.85809994e-01]
(50,)


We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [43]:
from keras.layers import SpatialDropout1D

In [44]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, weights=[emb], trainable=False, dropout=0.2),
#    Embedding(vocab_size, 50, input_length=seq_len, weights=[emb], dropout=0.2),
    Dropout(0.2),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  


In [45]:
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
model.fit(trn, y_train, validation_data=(test, y_test), batch_size=batch_size, epochs=2, verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
130s - loss: 0.6973 - acc: 0.5104 - val_loss: 0.6918 - val_acc: 0.5432
Epoch 2/2
128s - loss: 0.6898 - acc: 0.5275 - val_loss: 0.6860 - val_acc: 0.5657


In [47]:
model.save_weights(model_path+'emb-trainable1.h5')

We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [48]:
model.layers[0].trainable = True

In [49]:
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
model.optimizer.lr=1e-4

In [51]:
model.fit(trn, y_train, validation_data=(test, y_test), batch_size=batch_size, epochs=4, verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
147s - loss: 0.6765 - acc: 0.5760 - val_loss: 0.6731 - val_acc: 0.5971
Epoch 2/4
142s - loss: 0.6503 - acc: 0.6187 - val_loss: 0.6235 - val_acc: 0.6918
Epoch 3/4
151s - loss: 0.5626 - acc: 0.7207 - val_loss: 0.4983 - val_acc: 0.7925
Epoch 4/4
149s - loss: 0.4695 - acc: 0.7912 - val_loss: 0.4188 - val_acc: 0.8263


As expected, that's given us a nice little boost. :)

In [52]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).